## Install Tez and efficientnet-pytorch impelmentation


In [ ]:
!pip install tez
!pip install efficientnet-pytorch

## What is Diabetic Retinopathy?
There are at least 5 things to spot on in order to know that a patient have diabetic retinopahy. Image credit 

![](https://sa1s3optim.patientpop.com/assets/images/provider/photos/1947516.jpeg)


- [Image source](https://www.eyeops.com/contents/our-services/eye-diseases/diabetic-retinopathy)

From quick investigations of the data (see various pictures below), Hemorrphages, Hard Exudates and Cotton Wool spots are quite easily observed. However, examples of Aneurysm or Abnormal Growth of Blood Vessels are hard to find in the data. Perhaps the latter two cases are important if we want to catch up human benchmnark using our model.

## Why [Tez](https://github.com/abhishekkrthakur/tez)?
I always found the learning curve of pytorch a bit complicated, **Tez (तेज़ / تیز)** aims to make  pytorch training easy and allow fast prototyping by keeping things as simple and as customizable as possible.

## Import What You Need


In [ ]:
import os
import albumentations
import pandas as pd
import numpy as np
import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn
from torch.nn import functional as F

from efficientnet_pytorch import EfficientNet
from sklearn import metrics, model_selection, preprocessing
import matplotlib.pyplot as plt
import cv2

SEED = 42
IMAGE_SIZE = 256


## Split Data

In [ ]:
dfx = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_train, df_valid = model_selection.train_test_split(
        dfx, test_size=0.1, random_state=SEED, stratify=dfx.diagnosis.values
)

df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

image_path = "../input/aptos2019-blindness-detection/train_images/"
train_image_paths = [os.path.join(image_path, x+".png") for x in df_train.id_code.values]
valid_image_paths = [os.path.join(image_path, x+".png") for x in df_valid.id_code.values]
train_targets = df_train.diagnosis.values
valid_targets = df_valid.diagnosis.values

## Visualize Data

In [ ]:
fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(np.unique(train_targets)):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(np.unique(train_targets)):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

        plt.imshow(image, cmap='gray')
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

## Model via Tez.Model


In [ ]:
class EyeModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.effnet = EfficientNet.from_pretrained("efficientnet-b4")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1792, num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None


## Augmentations


In [ ]:

train_aug = albumentations.Compose([
            albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
            albumentations.Normalize(
                mean=[0.485], 
                std=[0.229], 
                max_pixel_value=255.0, 
                p=1.0
            )])
        
valid_aug = albumentations.Compose([
            albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
            albumentations.Normalize(
                mean=[0.485], 
                std=[0.229], 
                max_pixel_value=255.0, 
                p=1.0
            )])

## Generate Training and Validation dataset


In [ ]:
train_dataset = ImageDataset(
    image_paths=train_image_paths,
    targets=train_targets,
    augmentations=train_aug,
)

test_dataset = ImageDataset(
    image_paths=valid_image_paths,
    targets=valid_targets,
    augmentations=valid_aug,
)


## Load, Train & Save Model


In [ ]:
model = EyeModel(num_classes=5)
es = EarlyStopping(
    monitor="valid_loss", model_path="model.bin", patience=5, mode="min"
)


In [ ]:
model.fit(
    train_dataset,
    valid_dataset=test_dataset,
    train_bs=32,
    valid_bs=64,
    device="cuda",
    epochs=10,
    callbacks=[es],
    fp16=True,
)


In [ ]:
model.save("model.bin")